Importação das bibliotecas necessárias

In [24]:
import pandas as pd
import requests
from collections import Counter

Instanciando as URL's e fazendo as requições de cada uma separadamente

In [34]:
urlProducts = "https://fakestoreapi.com/products"
urlUsers = "https://fakestoreapi.com/users"
urlCarts = "https://fakestoreapi.com/carts"

response_products = requests.get(urlProducts)
response_users = requests.get(urlUsers)
response_carts = requests.get(urlCarts)

Fazendo a validação da requisição de cada API separadamente

In [35]:
response_products = requests.get(urlProducts)
if response_products.status_code == 200:
    products = response_products.json()
else:
    print(f"Falha na requisição dos produtos: {response_products.status_code}")
    products = []

response_users = requests.get(urlUsers)
if response_users.status_code == 200:
    users = response_users.json()
else:
    print(f"Falha na requisição dos usuários: {response_users.status_code}")
    users = []

response_carts = requests.get(urlCarts)
if response_carts.status_code == 200:
    carts = response_carts.json()
else:
    print(f"Falha na requisição dos carrinhos: {response_carts.status_code}")
    carts = []

Faz a converção das respostas para DataFrame ("CSV") do pandas se estiverem válidas

In [36]:
dfUsers = pd.DataFrame(users)
dfCarts = pd.DataFrame(carts)
dfProducts = pd.DataFrame(products)

Ajustando o campo de data para o formato ano, mês e dia

In [37]:
dfCarts['date'] = pd.to_datetime(dfCarts['date'])
dfCarts['date'] = dfCarts['date'].dt.strftime('%Y-%m-%d')

In [38]:
dfCarts.head(10)

,id,userId,date,products,__v
0,1,1,2020-03-02,"[{'productId': 1, 'quantity': 4}, {'productId'...",0
1,2,1,2020-01-02,"[{'productId': 2, 'quantity': 4}, {'productId'...",0
2,3,2,2020-03-01,"[{'productId': 1, 'quantity': 2}, {'productId'...",0
3,4,3,2020-01-01,"[{'productId': 1, 'quantity': 4}]",0
4,5,3,2020-03-01,"[{'productId': 7, 'quantity': 1}, {'productId'...",0
5,6,4,2020-03-01,"[{'productId': 10, 'quantity': 2}, {'productId...",0
6,7,8,2020-03-01,"[{'productId': 18, 'quantity': 1}]",0


Função para buscar a categoria mais frequente dentro do DataFrame de Produtos

In [39]:
def find_top_category(product_ids, df_products):
    product_categories = df_products[df_products['id'].isin(product_ids)]['category']
    top_category = Counter(product_categories).most_common(1)
    return top_category[0][0] if top_category else None

Estrutura para armazenar os dados processados

In [40]:
user_data = []

Itera sobre cada usuário para buscar as informações necessárias. Busca o ID do usuário, filtra os carrinhos do usuário, verifica se há carrinhos, encontra a data mais recente, obtem os ID's dos produtos, encontra a categoria mais recente e armazena os dados processados

In [41]:
for user in dfUsers.itertuples():
    user_id = user.id
    user_carts = dfCarts[dfCarts['userId'] == user_id]

    if not user_carts.empty:
        latest_date = user_carts['date'].max()

        product_ids = [product['productId'] for products_list in user_carts['products'] for product in products_list]
        top_category = find_top_category(product_ids, dfProducts)

        user_data.append({
            'user_id': user_id,
            'latest_date': latest_date,
            'top_category': top_category
        })

Criação do DataFrame a partir dos dados processados

In [42]:
df_user_data = pd.DataFrame(user_data)

df_user_data.head()

,user_id,latest_date,top_category
0,1,2020-03-02,men's clothing
1,2,2020-03-01,men's clothing
2,3,2020-03-01,jewelery
3,4,2020-03-01,electronics
4,8,2020-03-01,women's clothing


Exibição da quantidade de resgitros em dfCarts e no novo df criado

In [43]:
print(f"\nTotal de registros em dfCarts: {len(dfCarts)}")
print(f"Total de registros em df_user_data: {len(df_user_data)}")


Total de registros em dfCarts: 7
Total de registros em df_user_data: 5


Definindo o "user_id" como index do DataFrame

In [44]:
df_user_data.set_index('user_id', inplace=True)

In [45]:
df_user_data.head()

,latest_date,top_category
user_id,,
1,2020-03-02,men's clothing
2,2020-03-01,men's clothing
3,2020-03-01,jewelery
4,2020-03-01,electronics
8,2020-03-01,women's clothing


Salvando o DataFrame em arquivo CSV

In [ ]:
df_user_data.to_csv('user_cart_data.csv')